In [11]:
from evoVAE.utils.datasets import MSA_Dataset
from evoVAE.utils.seq_tools import encode_and_weight_seqs
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np 

ModuleNotFoundError: No module named 'evoVAE.utils.seq_tools'

## Data loading and preprocessing

In [8]:

# define threshold for sequence similarity 
THETA = 0.2

# Read in the datasets and create train and validation sets 
alns: pd.DataFrame = st.read_aln_file("../data/alignments/tiny.aln")
train, val = train_test_split(alns, test_size=0.2)


# create one-hot encodings and calculate reweightings 

# TRAINING 
train_encodings, train_weights = st.encode_and_weight_seqs(train["sequence"],theta=THETA)
train_ids = train["id"].values
train_dataset = MSA_Dataset(train_encodings, train_weights, train_ids)

# VALIDATION
val_encodings, val_weights = st.encode_and_weight_seqs(val["sequence"], THETA)
val_ids = val["id"].values
val_dataset = MSA_Dataset(val_encodings, val_weights, val_ids)


NameError: name 'st' is not defined